# Appunti sulla tesi *ottimizzazione degli algoritmi di ordinamento utilizzando AVX512*

# Premesse:
- Tutte le prove vengono eseguite sul processore AMD Ryzen 5 *7640U*
- I computer utilizzato è un framework 13 con 16Gb di RAM con velocità 5600 MT/s
- Il codice è compilato utilizzando sempre la flag `-march=znver5` per sfruttare tutte le potenzialità del processore
- Kernel: 6.11.11-1-MANJARO 

## Leggiamo le istruzioni ASM di bubbleSort e selectionSort compilate con varie opzioni:
Per comodità utilizzo [godbolt](https://godbolt.org/) per leggere le istruzioni ASM

## BubbleSort:

In [ ]:
void bubbleSort(double * v, int n) {
    for(int i = n - 1 ; i >= 0 ; i--) {
        for(int j = 0 ; j < i ; j++) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
        }
    }
}

- senza parametri (standard -O0): utilizza i registri principali (rax,rdx,rcx) e traduce ~ 1:1 il codice in c, lungo circa 200 istruzioni
- -O1: analogo a prima, ma lungo circa 30 istruzioni
- -O2: come con `-O1` ma utilizza i registri xmm e l'istruzione `pshufd` per velocizzare le comparazioni, circa 23 istruzioni
- -O3: identico a `-O2`, nessuna differenza
- aggiungendo **#pragma GCC target("avx512f,avx512dq,avx512cd,avx512bw,avx512vl,avx512vbmi,avx512ifma,avx512pf,avx512er,avx5124fmaps,avx5124vnniw,avx512bitalg,avx512vp2intersect") #include <immintrin.h>** per cercare di spingere l'utilizzo di AVX512 i risultati non cambiano per `-O1, -O2, -O3`


# SelectionSort

In [ ]:
void selectionSort(double * v, int dim) {
    for(int i = 0 ; i < dim ; i++) {
        int idx = i;
        for(int j = i + 1 ; j < dim ; j++) {
            if(v[j] < v[idx]) {
                idx = j;
            }
        }
        swap(v[i],v[idx]);
    }
}

- Senza parametri (standard -O0): utilizza i registri principali e traduce ~1:1, ~80 istruzioni
- -O1: utilizza anche registri come r9/r10/r11, sempre 1:1, ~50 istruzioni
- -O2: come `-O1` ma ~30 istruzioni
- -O3: identico a `-O2`, nessuna differenza
- aggiungendo **#pragma GCC target("avx512f,avx512dq,avx512cd,avx512bw,avx512vl,avx512vbmi,avx512ifma,avx512pf,avx512er,avx5124fmaps,avx5124vnniw,avx512bitalg,avx512vp2intersect") #include <immintrin.h>** per cercare di spingere l'utilizzo di AVX512 i risultati non cambiano per `-O1, -O2, -O3`

# CountingSort
```
void countingSort(int * v, int n) {
    int a = v[0];
    int b = v[0];
    for(int i = 0 ; i < n ; i++) {
        a = std::max(a,v[i]);
        b = std::min(b,v[i]);
    }
    int counts[a-b+1];
    for(int i = 0 ; i < a-b+1 ; i++) {
        counts[i] = 0;
    }
    for(int i = 0 ; i < n ; i++) {
        counts[v[i]-b]++;
    }
    int idx = 0, i = 0;
    while(idx < a-b+1) {
        if(counts[idx] != 0) {
            v[i++] = idx+b;
            counts[idx]--;
        } else {
            idx++;
        }
    }
}
```
*inserito il limite per ogni variabile a 240000*
- Senza parametri (-O0): utilizza i registri principali e traduce ~1:1, ~150 istruzioni
- -O1: utilizza nache r8/r9, ~80 istruzioni
- -O2: utilizza anche r10+, ~90 istruzioni
- -O3: utilizza i registri xmm, ~170 istruzioni

## tabella tempi, utilizzata la libreria fatta da Sansone(?) "timer.cpp"

tempi presi in maniera non rigorosa, utilizzati per fare una prima scrematura, su un array di 1000 valori interi generati con funzione rand(), massimo INT_MAX, con 1000 prove

|parametri | bubbleSort | selectionSort | insertionSort | countingSort | 
|---|---|---|---|---|
| default (-O0) | 1125±283 | 534±195 | 757±230 |
| -O1 | 349±150 | 275±114 | 454±157 |
| -O2 | 1558±264 | 275±116 | 380±156 |
| -O3 | 1559±256 | 279±122 | 441±178 |
| -O0 + pragma | 1118±247 | 545±202 | 406±156 | 756±210 |
| -O1 + pragma| 337±135 | 276±114 | 90±41 | 469±152 |
| -O2 + pragma| 372±160 | 275±109 | 79±36 | 376±132 |
| -O3 + pragma| 370±156 | 276±118 | 76±34 |456±166 |

# Utilizzando intrinsics
Come visto precedentemente il compilatore sfrutta i registri AVX solo per velocizzare le comparazioni tra 2 elementi, non sfruttando le potenzialità delle istruzioni SIMD.
Adesso scrivo il codice con le intrinsics SIMD prese da [Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html)

# SelectionSort
Iniziamo dall'algoritmo che promette un miglioramento maggiore
Da ora in avanti tutti i test sono fatti con parametri -O1, -O2 e -O3 perchè -O0 dà sempre SIGSEGV alle versioni vettorizzate (perchè era presente una load_ e non loardu_, adesso è stato sistemato)

Ho scritto due versioni del selectionSort ricercando le funzioni necessarie su [Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html) per utilizzare istruzioni AVX512.

La prima versione è molto simile al selectionSort senza istruzioni SIMD ma fa paragoni di 8 variabili alla volta e se trova un nuovo minimo cerca elemento per elemento a quale indice si trova.

La seconda versione ha la stessa struttura, ma riduce il tempo di esecuzione evitando di fare un ciclo per trovare l'indice, ma utilizzando un algoritmo branchless che sfrutta istruzioni SIMD.

Per trovare gli indici esegue una maskz_abs (non ho trovato nessuna funzione che eseguisse soltato una maschera) lasciando nel registro *c* solo gli indici dei valori minimi correnti. A questo punto mi basta uno qualsiasi dei valori diversi da zero rimasti nel registro *c*, utilizzo quindi una reduce_max che restituisce il massimo.

Mi aspetto che esistano intrinsics più efficienti, ma al momento è comunque più che sufficiente.

In [ ]:
void selectionSortAVX512_v1(double * v, int dim) {
    __m512d arr;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double lastMin = minimum;
        int idx = i;
        int j = i;
        while(j <= dim - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            minimum = min(minimum,_mm512_reduce_min_pd(arr));
            if(minimum != lastMin) {
                lastMin = minimum;
                for(int k = 0 ; k < 8 ; k++) {
                    if(v[j+k] == minimum) {
                        idx = j+k;
                        break;
                    }
                }
            }
            j += 8;
        }
        // use regular code to check the last values with index not multiple of 8
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

In [ ]:
void selectionSortAVX512_v2(double * v, int dim) {
    __m512d arr,min_vect;
    long idxs_arr[8] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_loadu_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double last = minimum;
        int idx = i;
        int j = i;
        while(j <= dim - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            minimum = min(minimum,_mm512_reduce_min_pd(arr));
            if(minimum != last) {
                last = minimum;
                min_vect = _mm512_set1_pd(minimum);
                mask = _mm512_cmpeq_pd_mask(arr,min_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                idx = j + _mm512_reduce_max_epi64(c);
            }
            j+=8;
        }
        // use regular code to check the last values with index not multiple of 8
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

# Considerazioni
Premessa

Ho testato le 3 versioni di *selectionSort* con array di dimensione 10,100,1000,10000 e 100000. Per ogni dimensione ho compilato sia con *-O1*, *-O2*, *-O3* (per tutti ho utilizzato *-march=znver5*). Per ognuno ho provato sia con la versione 0(senza SIMD),1 e 2 dell'algoritmo. Ogniuno è stato testato con array in ordine casuale, crescente e decrescente. Per ogni combinazione ho eseguito 20 volte per avere un sample size minimo.

## Osservazioni 
- La flag *-O1* è quella che rende la versione *v0* $\textrm{consistentemente}^{1}$ più rapido
- La flag *-O3* è quella che rende la versione *v1* $\textrm{consistentemente}^{1}$ (ma minimamente) più lento
- Non ci sono sostanziali differenze tra *-O1*, *-O2* e *-O3* utilizzando la versione *v2*
- Per array di dimensione 10 i valori sono troppo ravvicinati per ricavarne considerazioni
- Per array di dimensione 100 si inizia a notare che il caso in cui i dati sono generati in ordine decrescente impiegano $\textrm{consistentemente}^{2}$ meno tempo di quelli in cui sono inseriti in ordine casuale, ma non si apprezza differenze tra le versioni dell'algoritmo
- A partire da array di dimensione 1000 si nota che qualsiasi versione risulta consistentemente più rapida se eseguita su array ordinati in modo decrescente che in modo casuale (ipotizzo sia dovuto alla branch prediction del processore)
- Per array di dimensione 1000  notiamo che *v1* è il 13,6%$^{3}$ più veloce di *v0*, mentre *v2* lo è il 31,6%$^{3}$.

Note:
1. Per ogni lunghezza testata e per ogni ordinamento iniziale dell'array
2. Per ogni lunghezza testata
3. compilando tutto con flag *-O1* e utilizzando la mediana dei tempi impiegati per ordinare un array di numeri generati casualmente


# Altre versioni
Ho deciso di scrivere le mie versioni dell'algoritmo prima di fare di cerche per evitare di essere influenzato da algoritmi altrui.
Cercando su internet non si trovano versioni di facile lettura, ma utilizzando i principali tool di assistenza alla programmazione possimamo ottenere alcuni spunti.
- *ChatGPT*, anche dopo alcuni $\textrm{solleciti}^{1}$, non restituisce un codice che esegue come richiesto.
- *Microsoft copilot* restituisce un codice che esegue come richiesto e che, sfruttando instrinsics di cui non ero a conoscenza.

Si osserva che la versione di *copilot* non necessita di un ciclo per gli ultimi massimo 7 valori, questo è dovuto alla prima condizione dell'if più interno.

Note:
1. Per solleciti sis intende messaggi in cui si precisa quale parte del codice correggere

In [ ]:
void selectionSortAVX512_copilot(double* arr, int n) {
    __m512d current_min_vals;
    __mmask8 mask;
    __m512d current_vals;
    for (int i = 0; i < n - 1; ++i) {
        int min_idx = i;
        double min_val = arr[i];

        for (int j = i + 1; j < n; j += 8) {
            __m512d current_vals = _mm512_loadu_pd(&arr[j]);
            __m512d current_min_vals = _mm512_set1_pd(min_val);
            __mmask8 mask = _mm512_cmplt_pd_mask(current_vals, current_min_vals);
            if (mask) {
                // Extract the new minimum value from current_vals
                for (int k = 0; k < 8; ++k) {
                    if (j + k < n && (mask & (1 << k))) {
                        if (arr[j + k] < min_val) {
                            min_val = arr[j + k];
                            min_idx = j + k;
                        }
                    }
                }
            }
        }

        // Swap the found minimum element with the first element
        if (min_idx != i) {
            std::swap(arr[i], arr[min_idx]);
        }
    }
}

In [ ]:
// per completezza
void selectionSortAVX512_ChatGPT(double* arr, size_t size) {
    for (size_t i = 0; i < size; ++i) {
        size_t min_index = i;
        double min_val = arr[i];

        // Find the minimum in the remaining array using AVX-512
        for (size_t j = i; j + 8 <= size; j += 8) {
            // Load 8 elements into an AVX-512 register
            __m512d reg = _mm512_loadu_pd(&arr[j]);

            // Compare and find the minimum value in the register
            __mmask8 cmp_mask = _mm512_cmp_pd_mask(reg, _mm512_set1_pd(min_val), _CMP_LT_OQ);

            // Update min_val and min_index directly using intrinsics
            if (cmp_mask) {
                // prende il primo, non funziona
                int first_true = _tzcnt_u32(cmp_mask); // Get the index of the first true comparison
                min_val = arr[j + first_true];
                min_index = j + first_true;
            }
        }

        // Check any remaining elements not fitting in an AVX-512 register
        for (size_t j = (size / 8) * 8; j < size; ++j) {
            if (arr[j] < min_val) {
                min_val = arr[j];
                min_index = j;
            }
        }

        // Swap the found minimum with the current element
        if (min_index != i) {
            std::swap(arr[i], arr[min_index]);
        }
    }
}

## Osservazioni
La versione *copilot* è sostanzialmente equivalente alla *v2*$^{1}$ per array di dimensione 1000, mentre al crescere della dimensione degli array aumenta il distacco a suo favore. Questo è dovuto al fatto che il codice tra il secondo for e l'if successivo è molto più efficiente dell'analoga parte di *v2*. Tuttavia la parte interna al suddetto if è più rapida nella versione *v2*.

Unendo le due versioni otteniamo la versione *v5*.

Note:
1. per array non ordinati

In [ ]:
void selectionSortAVX512_v5(double * v, int dim) {
    __m512d arr,min_vect;
    long idxs_arr[8] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_loadu_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double last = minimum;
        int idx = i;
        int j = i + 1;
        min_vect = _mm512_set1_pd(minimum);
        while(j < dim -8) {
            arr = _mm512_loadu_pd(&v[j]);
            mask = _mm512_cmplt_pd_mask(arr, min_vect);
            if(mask) {
                minimum = _mm512_reduce_min_pd(arr);
                min_vect = _mm512_set1_pd(minimum);
                mask = _mm512_cmpeq_pd_mask(arr,min_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                idx = j + _mm512_reduce_max_epi64(c);
            }
            j+=8;
        }
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

Con array di lunghezza 1000 questa versione compilata con la flag *-O1* risulta la più lenta con uno scarto del 6.8%, tuttavia all'aumentare della lunghezza la differenza diventa sempre più insignificante.

Su array di dimensione 1000 questa versione è il 29.2%$^{2}$ più veloce delle versioni *v2* e *copilot*, mentre è il 51.7% più veloce della versione *v0*

All'aumentare della dimensione la versione *copilot* si avvicina sempre di più, perché la parte di codice che le distingue viene eseguita meno $\textrm{volte}^{3}$

Note:
1. Su array con valori casuali
2. Statistica calcolata su 100 esecuzioni con array di elementi casuali casuali
3. Possiamo notarlo anche con la differenza tra le versioni *v1* e *v2*

eseguendo un test estensivo ho notato una estrema differenza tra le statistiche degli algoritmi compilati con la flag *-O1* e gli altri(quasi il doppio nel tempo di esecuzione), quindi ho deciso di eseguire nuovamente i test ma lasciando al computer più spazio per aspirare l'aria, sperando sia sufficiente. Altrimenti sarò costtretto ad eseguire un carico molto elevato prima in modo da avere il processore in thermal protection durante tutti i test.
pare abbia trovato un equilibrio intorno ai 65/66 gradi

Purtroppo avere più spazio non è stato sufficiente, quindi ho deciso di eseguire test solo sulle parti che voglio comparare al momento

## Analisi compilato

Analizziamo adesso come viene compilato il codice, per qeusta analisi utilizzerò la flag -O1, poichè le intrinsics vengono compilate nello stesso modo per tutti i livelli di ottimizzazione (da controllare).


|riga c++|intrinsic|riga asm|istruzione asm|significato|
|------|------|------|------|------|
|4|_mm512_loadu_epi64|26|vmovdqa64 zmm0, addr| move data from address to register|
|11|_mm512_set1_pd|278|vmovsd xmm4,addr|move data from address to register|
|11|_mm512_set1_pd|285|vbroadcastsd zmm1,xmm4|extend data from register to register|
|13|_mm512_loadu_pd|61|vmovupd zmm0,addr|move data from address to register|
|14|_mm512_cmplt_pd_mask|68|vcmpltpd k0,zmm0,zmm1| compare less-than and output in mask|
|16|_mm512_reduce_min_pd|81|vextractf64x4 ymm2, zmm0, 1| set ymm2[i] = zmm0[i+4] for i=0..3|
|16|_mm512_reduce_min_pd|88|vminpd ymm2, ymm2, ymm0 | set ymm2[i] = min(ymm2[i],ymm0[i])|
|16|_mm512_reduce_min_pd|92|vextractf64x2 xmm1, ymm2, 1 |set xmm1[i] = ymm2[i+2] for i=0,1|
|16|_mm512_reduce_min_pd|99|vminpd xmm1, xmm1, xmm2|set xmm1[i] = min(xmm2[i],xmm1[i])|
|17|_mm512_set1_pd|113|vbroadcastsd zmm1, xmm1|extend data from register to register|
|18|_mm512_cmpeq_pd_mask|119|vcmpeqpd k1, zmm0, zmm1|compare equal and output mask|
|19|_mm512_maskz_abs_epi64|126|vpabsq zmm0 {k1} {z}, zmm3| zmm0 = abs(zmm3[i]) if k1[i] for i=0..7|
|20|_mm512_reduce_max_epi64|132|vshufi64x2 zmm2, zmm0, zmm0, 0x4e|~ set ymm2[i] = zmm0[i+4] for i=0..3|
|20|_mm512_reduce_max_epi64|139|vpmaxsq zmm0, zmm0, zmm2|zmm0[i] = max(zmm0[i],zmm2[i]) for i=0..3|
|20|_mm512_reduce_max_epi64|145|vshufi64x2 zmm2, zmm0, zmm0, 0xb1| ~ move part of zmm0 to zmm2|
|20|_mm512_reduce_max_epi64|152|vpmaxsq zmm0, zmm0, zmm2|zmm0[i] = max(zmm0[i],zmm2[i]) for i=0..3|
|20|_mm512_reduce_max_epi64|158|vpermq zmm2, zmm0, 0xb1|~ zmm2[0] = zmm0[1], zmm2[1] = zmm0[0], ...|
|20|_mm512_reduce_max_epi64|165|vpmaxsq zmm0, zmm0, zmm2|zmm0[i] = max(zmm0[i],zmm2[i]) for i=0..3|

Osservazioni:
- la maggior parte delle istruzioni SIMD di questa funzione solo solo per _mm512_reduce_min_pd e _mm512_reduce_max_epi64, entrambe restutuiscono il valore minimo(massimo) di tipo double(long) del registro vettorizzato.
- tutte le altre funzioni SIMD sono eseguite in una sola istruzione (set1 utilizza una istruzione per spostare il valore dalla memoria a un registro, la parte di estensione da registro a registro vettorizzato è una sola istruzione)

# Possibile miglioramento
Ho scitto una funzione **isSortedAVX512_v1** ed ho provato sia a chiamarla all'inizio di ogni ciclo interno sia ad inserirla all'interno dell'esecuzione normale.
Degli algoritmi considerati nessuno rende considerevolmente più ordinato l'array dopo ogni iterazione quindi mi aspetto che questa modifica non porti miglioramenti in nessun algoritmo.

Comparando queste versioni con v5 si nota (come atteso) che sono considerevolmente più lente all'aumentare della lunghezza. Questo è dovuto al fatto che per ogni iterazione di `i` eseguo una lettura in più del resto dell'array.

# InsertionSort
Ho implementato una versione dell'insertionSort e fatto generare una versione da Chat-GPT e una da microsoft copilot. Delle due IA solo la seconda sembra essere riuscia a produrre un codice con qualche utilità (non sempre funziona, ritengo di poterlo sistemare nei prossimi giorni).

In [ ]:
void insertionSortAVX512_v1(double * v, int dim) {
    for (int i = 1; i < dim; ++i) {
        int j = i-1;
        double key = v[i];
        while(j >= 8 && v[j-8] > key) {
            __m512d vec = _mm512_loadu_pd(&v[j-7]);
            _mm512_storeu_pd(&v[j-6],vec);
            j -= 8;
        }
        while (j >= 0 && v[j] > key) {
            v[j + 1] = v[j];
            j--;
        }
        v[j + 1] = key;
    }
}

L'idea alla base è spostare 8 elementi alla volta se il valore corrente va spostato di almeno 8 elementi, altrimenti eseguo il classico ciclo senza istruzioni SIMD.

# Osservazioni
Nè Chat-GPT nè Microsoft copilot sono state in grado di fornire una versione che funzionasse.
Questo algoritmo utilizza solamente 2 istruzioni SIMD poichè non c'è altro da vettorizzare (il confronto è sufficiente con l'elemento di 8 pposizioni precedente).

Constatiamo che questo algoritmo sia in partenza (nella versione senza istruzioni SIMD) più efficiente di SelectionSort per il fatto che per le iterazioni interne terminano appena trovato un valore minore del corrente.
Notiamo che la versione SIMD di questo algoritmo sfrutta solamente la $\textrm{load}^1$ e la $\textrm{store}^1$ rispetto alle molteplici utilizzate dal SelectionSort, questo perchè non si ha bisogno di calcoli articolati, ma solo di spostare i valori nel caso la comparazione dia esito positivo.

Eseguiamo il codice e $\textrm{confrontiamolo}^2$ con i tempi di esecuione del selectionSort:

|Dimensione|SelectionSort_v0|SelectionSort_v5|Miglioramento %|InsertionSort_v0|InsertionSort_v2|Miglioramento %| Miglioramento SelV5/InsV2|
|-----|-----|-----|-----|-----|-----|-----|-----|
| 1000 |276 | 127 |54 |111 |40 |64 |68|
| 10000|15'399|5'287|66|8'973|2'860|68|46|
| 100000|1'392'983|435'025|69|796'252|262'530|67|40|
| 200000 $\textrm{ }^3$|5'544'907|1'856'801|67|3'219'257|1'054'202|67|43|

E' evidente la differenza



Note:
1. rispettivamente *_mm512_loadu_pd* per caricare i valori nei registri e *_mm512_storeu_pd* per spostare i valori in memoria
2. utilizzando sempre la flag O1
3. Purtroppo aumentare di un altro ordine di grandezza non è possibile perchè rende la dimensione troppo grande, quindi mi limito a raddoppiare

# Analisi compilato
Come sottolineato precedentemente, il numero di istruzioni SIMD è nullo, quindi è possibile solo notare che il codice asm è una fedele trasposizione dal c++ in modo diretto

# bubbleSort
Analogamente al selectionSort ho implementato 2 versioni del bubbleSort più una in cui il ciclo interno è branchless

In [ ]:
void bubbleSortAVX512_v1(double * v, int dim) {
    __m512d arr;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            if(maximum != v[j+7]) {
                for(int k = 0 ; k < 8 ; k++) {
                    if(v[j+k] == maximum) {
                        std::swap(v[j+7],v[j+k]);
                        break;
                    }
                }
            }
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    } 
    t.stop();
}

In [ ]:
void bubbleSortAVX512_v2(double * v, int dim) {
    __m512d arr,max_vect;
    long idxs_arr[16] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_loadu_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            if(maximum != v[j+7]) {
                max_vect = _mm512_set1_pd(maximum);
                mask = _mm512_cmpeq_pd_mask(arr,max_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                int idx = j + _mm512_reduce_max_epi64(c);
                std::swap(v[idx],v[j+7]);
            }
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    }
}

In [ ]:
void bubbleSortAVX512_v3(double * v, int dim) {
    __m512d arr,max_vect;
    long idxs_arr[16] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_loadu_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            max_vect = _mm512_set1_pd(maximum);
            mask = _mm512_cmpeq_pd_mask(arr,max_vect);
            c = _mm512_maskz_abs_epi64(mask,idxs_vect);
            int idx = j + _mm512_reduce_max_epi64(c);
            std::swap(v[idx],v[j+7]);
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    }
}

Ho deciso di implementare il bubbleSort invertendo l'elemento di valore (tra gli 8 confrontati) maggiore con quello nel registro con indice più alto in ogni passata del ciclo interno.
Analogamente al selectionSort ho scritto una prima versione che utilizza un ciclo for e una che utilizza le istruzioni SIMD per trovare l'indice del valore massimo.
Ho scritto anche una terza versione che sposta a priori il massimo all'indice massimo del vettore, il ciclo interno è branchless ma sicuramente risulterà più lento.

# Osservazioni
- Anche qui nè Chat-GPT nè Microsoft copilot sono state in grado di restituire del codice utile. 
- Il codice risultante è estremamente simile a quello del selectionSort

Tabella dei tempi (presi con flag O1 e array di numeri generati casualmente)
|Dimensione|v0|v1|v2|Peggioramento v0/v1|Peggioramento v0/v2|Peggioramento v1/v2|
|--|--|--|--|--|--|--|
|1000|454|491|662|-8.15%|-37.0%|-26.68%|
|10000|33'433|42'219|56'759|-26.28%|-69.77%|-34.44%|
|100000|4'020'427|4'122'256|5'559'596|-2.53%|-38.28%|-34.87%|

Come si evince dalla tabella più andiamo ad utilizzare istruzioni SIMD più l'lagoritmo è lento. Misurando le differenze di tempo all'interno dell'if tra v1 e v2 vediamo che v2 ha un ciclo interno più rapido di v1. Leggendo il codice c++ notiamo anche che quella è l'unica parte in cui le due versioni differiscono. Questo significa che deve esserci una differenza nel compilato tra le due versioni (oltre all'ovvia differenza tra le parti di codice diverse anche tra le parti di codice uguali).

Ho confrontato anche cambiando i livelli di ottimizzazione (O0,O1,O2 ed O3) e questa caratteristica rimane costante, analizziamo dunque i due compilati (con flag O1):

In [ ]:
bubbleSortAVX512_v1(double*, int):
        mov     r9d, esi				|
        mov     r8d, esi				|
        dec     r8d					|-> inizio funzione
        js      .L11					|
        sub     r9d, 9				|
        jmp     .L13					|
.L36:
        vmovsd  QWORD PTR [r10], xmm1
        vmovsd  QWORD PTR [rdx], xmm2
.L14:
        add     eax, 7
        add     rcx, 56
        cmp     eax, r9d
        jge     .L35
.L19:
        mov     r10, rcx
        vmovupd zmm0, ZMMWORD PTR [rcx-56]		|
        vextractf64x4   ymm1, zmm0, 0x1			|
        vmaxpd  ymm1, ymm1, ymm0			|
        vextractf64x2   xmm0, ymm1, 0x1			|
        vmaxpd  xmm0, xmm0, xmm1			|-> loadu + reduce_max
        vpermilpd       xmm1, xmm0, 1			|
        vmaxpd  xmm0, xmm0, xmm1			|
        vmovsd  xmm2, QWORD PTR [rcx]			|
        vucomisd        xmm2, xmm0	        	|
        jp      .L28		                        |
        je      .L14			|-> continuo il for interno
.L28:
        mov     rdx, rcx					|
        lea     rsi, [rcx-64]					|
.L18:
        vmovsd  xmm1, QWORD PTR [rdx]	        		|
        vucomisd        xmm1, xmm0				|
        jp      .L16		|-> 
        je      .L36			|-> continuo il for interno
.L16:
        sub     rdx, 8	        |
        cmp     rdx, rsi	|
        jne     .L18		|-> ciclo for per trovare il massimo degli 8
        jmp     .L14			|-> continuo il for interno
.L35:
        cmp     eax, r8d
        jge     .L20			|-> parte finale ciclo for esterno
.L26:
        cdqe
        jmp     .L23
.L21:
        inc     rax
        cmp     r8d, eax
        jle     .L20
.L23:
        vmovsd  xmm0, QWORD PTR [rdi+rax*8]		|
        vmovsd  xmm1, QWORD PTR [rdi+8+rax*8]		|
        vcomisd xmm0, xmm1				|
        jbe     .L21				        |-> ciclo while SISD
        vmovsd  QWORD PTR [rdi+rax*8], xmm1		|
        vmovsd  QWORD PTR [rdi+8+rax*8], xmm0		|
        jmp     .L21
.L37:
        ret
.L20:
        dec     r8d					|
        dec     r9d					|
.L13:						        |-> parte finale del ciclo for esterno
        lea     rcx, [rdi+56]		                |
        mov     eax, 0			        	|
        cmp     r8d, 8				        |
        jg      .L19			                |-> torno alla parte della funzione giusta in base alla lunghezza rimasta
        test    r8d, r8d				|
        jle     .L37					|
        mov     eax, 0			        	|
        jmp     .L26					|
.L11:
        ret


In [ ]:
bubbleSortAVX512_v2(double*, int):
        mov     r9d, esi										|
        mov     r8d, esi										|
        dec     r8d											|
        js      .L38											|-> inizio funzione
        sub     r9d, 9										        |
        vmovdqa64       zmm3, ZMMWORD PTR .LC0[rip]			                                |
        jmp     .L40											|
.L52:
        vbroadcastsd    zmm0, xmm0							|
        vcmppd  k1, zmm2, zmm0, 0							|
        vpabsq  zmm0{k1}{z}, zmm3							|
        vshufi64x2      zmm2, zmm0, zmm0, 78				        	|
        vpmaxsq zmm0, zmm0, zmm2							|
        vshufi64x2      zmm2, zmm0, zmm0, 177			                	|
        vpmaxsq zmm0, zmm0, zmm2							|-> set1+cmpeq+mask+reduce_max
        vpermq  zmm2, zmm0, 177								|
        vpmaxsq zmm0, zmm0, zmm2							|
        vmovq   rax, xmm0								|
        add     eax, edx								|
        cdqe										|
        lea     rax, [rdi+rax*8]							|
        vmovsd  xmm0, QWORD PTR [rax]						        |
        vmovsd  QWORD PTR [rax], xmm1   						|
        vmovsd  QWORD PTR [rsi], xmm0	        					|
.L41:
        add     edx, 7			        |
        add     rcx, 56			        |
        cmp     edx, r9d			|-> gestione while SIMD
        jge     .L57				|
.L43:
        mov     rsi, rcx			        	|
        vmovupd zmm2, ZMMWORD PTR [rcx-56]			|
        vextractf64x4   ymm1, zmm2, 0x1				|
        vmaxpd  ymm1, ymm1, ymm2				|
        vextractf64x2   xmm0, ymm1, 0x1				|
        vmaxpd  xmm0, xmm0, xmm1				|-> loadu + reduce_max
        vpermilpd       xmm1, xmm0, 1				|
        vmaxpd  xmm0, xmm0, xmm1				|
        vmovsd  xmm1, QWORD PTR [rcx]				|
        vucomisd        xmm1, xmm0			        |
        jp      .L52						|
        je      .L41						|
        jmp     .L52						|
.L57:
        cmp     edx, r8d			|
        jge     .L44				|-> gestion while SISD
.L50:
        movsx   rax, edx
        jmp     .L47
.L45:
        inc     rax
        cmp     r8d, eax
        jle     .L44
.L47:
        vmovsd  xmm0, QWORD PTR [rdi+rax*8]		                |
        vmovsd  xmm1, QWORD PTR [rdi+8+rax*8]	                        |
        vcomisd xmm0, xmm1						|
        jbe     .L45							|-> while SISD
        vmovsd  QWORD PTR [rdi+rax*8], xmm1		                |
        vmovsd  QWORD PTR [rdi+8+rax*8], xmm0	                        |
        jmp     .L45
.L58:
        ret
.L44:
        dec     r8d
        dec     r9d
.L40:
        lea     rcx, [rdi+56]		                |
        mov     edx, 0				        |
        cmp     r8d, 8				        |
        jg      .L43					|
        test    r8d, r8d				|-> gestione for interno
        jle     .L58					|
        mov     edx, 0				        |
        jmp     .L50					|
.L38:
        ret
.LC0:
        .quad   0		|
        .quad   1		|
        .quad   2		|
        .quad   3		|
        .quad   4		|-> numeri in memoria per gli indici dei registri zmm
        .quad   5		|
        .quad   6		|
        .quad   7		|

Le differenze tra i due compilati non sembrano spiegare come la versione che utilizza più istruzioni SIMD sia più lenta dell'altra.

Considerando che durante l'esecuzione del bubbleSort la parte non ordinata viene parzialmente ordinata ma la versione 1 di questo algoritmo cicla fino a trovare il massimo il numero di volte che viene eseguito il for (con variabile k) è decisamente maggiore della media (~4.4 per gli array di 100 e ~7.6 con quelli di 1000 elementi). Quindi non è la parte interna all' `if(maximum != v[j+7]) {` a renderlo più veloce

Partiamo dal fatto che la `reduce_max` è la parte più onerosa della funzione, per ridurne l'utilizzo possiamo confrontare il vettore corrente con il numero maggiore trovato nell'ultiumo ciclo. Sarà molto probabile che il massimo precedente sia maggiore di tutti i valori del vettore corrente quindi gestire questo caso senza `reduce_max` contribuisce molto al miglioramento. Possiamo anche controllare se il valore all'indice massimo è il massimo corrente, nel qual caso non necessiteremo della funzione `swap`.

Questo ultimo caso risulterà molto più frequente rispetto al trovare il massimo del vettore corrente in qualsiasi altra posizione, poichè i vettori sono sempre tutti allineati tra loro ed il massimo corrente viene spostato all'indice corrente maggiore.

In [ ]:
void bubbleSortAVX512_v6(double * v, int dim) {
    __m512d arr, curr;
    __mmask8 mask;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        int j = 1;
        curr =  _mm512_set1_pd(v[0]);
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            mask = _mm512_cmplt_pd_mask(curr,arr);
            if(!mask) {
                std::swap(v[j+7],v[j-1]);
            } else {
                curr = _mm512_set1_pd(v[j+7]);
                mask = _mm512_cmplt_pd_mask(curr,arr);
                if(mask) {
                    double maximum = _mm512_reduce_max_pd(arr);
                    curr = _mm512_set1_pd(maximum);
                    mask = _mm512_cmpeq_pd_mask(arr,curr);
                    int idx = _tzcnt_u32(mask) + j;
                    std::swap(v[j+7],v[idx]);
                }
            }
            j+=8;
        }
        while(j <= i ) {
            if(v[j-1] > v[j]) {
                std::swap(v[j-1],v[j]);
            }
            j++;
        }
    }
}

Questa versione ci dà un miglioramento che va dal 60% con array di dimensione 1000 all'80% con array di dimensione 100000. 

# O1 vs O2 vs O3

O3 non garantisce l'equivalenza a O2 e O1, ma è il più veloce.
O2 dovrebbe essere il più veloce ma (almeno) con selectionSort v8 (nella tesi v3) è più lento (del 26%) di O1.